In [320]:
import numpy as np
import pandas as pd
from bayes_opt import BayesianOptimization
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_percentage_error, mean_squared_error
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split

In [321]:
# Load Dataset
X_train = pd.read_csv("dataset_5yrs_xTrain.csv")
X_remain = pd.read_csv("dataset_5yrs_xTest.csv")
y_train = pd.read_csv("dataset_5yrs_yTrain.csv")
y_remain = pd.read_csv("dataset_5yrs_yTest.csv")

In [322]:
# Split X_remain into X_validate and X_test
X_validate, X_test, y_validate, y_test = train_test_split(X_remain, y_remain, test_size=0.5)

In [323]:
# Variable Correlations
train_df = pd.concat([y_train, X_train], axis=1)
corr_matrix = train_df.corr(method='pearson')
corr_matrix["no_of_accidents"].sort_values(ascending=False)

no_of_accidents                            1.000000
number_of_vehicles                         0.843779
number_of_casualties                       0.747080
Thursday                                   0.260678
Tuesday                                    0.253360
Sunday                                     0.233905
Friday                                     0.230826
Wednesday                                  0.229776
Saturday                                   0.228003
Monday                                     0.221901
location                                   0.052231
month                                      0.013590
accident_severity                          0.009733
trunk_road_flag                           -0.018806
carriageway_hazards                       -0.022832
urban_or_rural_area                       -0.024194
special_conditions_at_site                -0.031251
pedestrian_crossing_human_control         -0.037582
speed_limit                               -0.045539
first_road_n

Regression Models

In [324]:
# Method for evaluating models
def print_metrics(y_pred, y_true):
    # Mean Squared Error
    mse = mean_squared_error(y_true, y_pred)
    print("Mean Squared Error (MSE): ", mse)

    # Root Mean Squared Error
    rmse = np.sqrt(mse)
    print("Root Mean Squared Error (RMSE): ", rmse)

    # R-squared
    r2 = r2_score(y_true, y_pred)
    print("R-squared (R2): ", r2)

    # Mean Absolute Percentage Error
    mape = mean_absolute_percentage_error(y_true, y_pred)
    print("Mean Absolute Percentage Error (MAPE): ", mape)

In [325]:
#Linear Regression
# Create a linear regression object
lr_model = LinearRegression()

# Fit the model to the training data
lr_model.fit(X_train, y_train)

# Make predictions on the validation set
y_predict = lr_model.predict(X_validate)

print_metrics(y_predict, y_validate)

Mean Squared Error (MSE):  0.0007531306364851331
Root Mean Squared Error (RMSE):  0.027443225693878137
R-squared (R2):  0.9983552304365708
Mean Absolute Percentage Error (MAPE):  0.017644798363835337


In [326]:
# Using Grid Search to improve LR model
#  Define a range of hyper-parameters to search over
param_grid = {"fit_intercept": [True, False]}

# Create a grid search object
grid_search = GridSearchCV(lr_model, param_grid, cv=5, scoring="neg_mean_squared_error")

# Fit the grid search object to the training data
grid_search.fit(X_train, y_train)

# Make predictions on the validation set using the best estimator
best_model = grid_search.best_estimator_
y_predict = best_model.predict(X_validate)

print_metrics(y_predict, y_validate)
# No difference

Mean Squared Error (MSE):  0.0007531306364851331
Root Mean Squared Error (RMSE):  0.027443225693878137
R-squared (R2):  0.9983552304365708
Mean Absolute Percentage Error (MAPE):  0.017644798363835337


In [327]:
# Using Randomized search to improve LR model
# Define a range of hyper-parameters to search over
param_dist = {"fit_intercept": [True, False], "copy_X": [True, False], "n_jobs": [1, -1],
              "positive": [True, False]}

# Create a randomized search object
random_search = RandomizedSearchCV(lr_model, param_distributions=param_dist, n_iter=16, cv=5,
                                   scoring="neg_mean_squared_error", random_state=42)

# Fit the randomized search object to the training data
random_search.fit(X_train, y_train)

# Make predictions on the validation set using the best estimator
best_model = random_search.best_estimator_
y_predict = best_model.predict(X_validate)

# Calculate the MSE of the predictions
print_metrics(y_predict, y_validate)
# No difference

Mean Squared Error (MSE):  0.0007531306364851331
Root Mean Squared Error (RMSE):  0.027443225693878137
R-squared (R2):  0.9983552304365708
Mean Absolute Percentage Error (MAPE):  0.017644798363835337


In [328]:
# Improving LR model using Bayesian Optimization

# Define the function to optimize
def optimize_linear_regression(fit_intercept, copy_x, positive):
    # Convert the continuous value of copy_X to a boolean
    fit_intercept = bool(round(fit_intercept))
    copy_x = bool(round(copy_x))
    positive = bool(round(positive))
    # Create a linear regression model with the given hyper-parameters
    model = LinearRegression(fit_intercept=fit_intercept, copy_X=copy_x, positive=positive)
    # n_jobs=n_jobs
    # Fit the model to the training data
    model.fit(X_train.values, y_train.values)
    # Calculate the mean squared error on the validation set
    mse = np.mean((model.predict(X_validate.values) - y_validate.values) ** 2)
    # Return the negative mean squared error (to be maximized by the optimizer)
    return -mse


# Define the hyperparameter space to optimize over
pbounds = {'fit_intercept': (0, 1), 'copy_x': (0, 1), 'positive': (0, 1)}
# 'n_jobs': (None, 1, -1),

# Create the optimizer object
optimizer = BayesianOptimization(f=optimize_linear_regression, pbounds=pbounds, random_state=42)

# Run the optimization for a given number of iterations
optimizer.maximize(init_points=5, n_iter=50)

# Get the best hyper-parameters found by the optimizer
best_fit_intercept = optimizer.max['params']['fit_intercept']
best_copy_X = optimizer.max['params']['copy_x']
# best_n_jobs = optimizer.max['params']['n_jobs']
best_positive = optimizer.max['params']['positive']

# Create a linear regression model with the best hyper-parameters and fit it to the data
best_model = LinearRegression(fit_intercept=bool(round(best_fit_intercept)), copy_X=bool(round(best_copy_X)),
                              positive=bool(round(best_positive)))
# n_jobs=best_n_jobs,
best_model.fit(X_train, y_train)

# Make predictions on the validation set using the best estimator
y_predict = best_model.predict(X_validate)

print_metrics(y_predict, y_validate)
# No difference

|   iter    |  target   |  copy_x   | fit_in... | positive  |
-------------------------------------------------------------
| 1         | -0.000753 | 0.3745    | 0.9507    | 0.732     |
| 2         | -1.712    | 0.5987    | 0.156     | 0.156     |
| 3         | -0.000753 | 0.05808   | 0.8662    | 0.6011    |
| 4         | -1.712    | 0.7081    | 0.02058   | 0.9699    |
| 5         | -1.712    | 0.8324    | 0.2123    | 0.1818    |
| 6         | -0.000753 | 0.0       | 1.0       | 1.0       |
| 7         | -0.000753 | 0.0       | 1.0       | 0.0       |
| 8         | -0.000753 | 1.0       | 1.0       | 1.0       |
| 9         | -0.000753 | 1.0       | 1.0       | 0.0       |
| 10        | -0.000753 | 1.0       | 1.0       | 0.496     |
| 11        | -0.000753 | 0.4455    | 1.0       | 0.2002    |
| 12        | -0.000753 | 0.0       | 1.0       | 0.4854    |
| 13        | -0.000753 | 0.5852    | 1.0       | 1.0       |
| 14        | -0.000753 | 0.0127    | 0.7015    | 0.9944    |
| 15    

Linear regression model has reached its maximum performance given the available data and the chosen hyper-parameters.

This is probably because the chosen hyper-parameters are already near-optimal and further tuning won't make a significant difference in the model's performance.